# Runtimes Cookbook

## Overview

In this tutorial, we will explore the `Runtimes` in CAMEL, focusing on three specific runtime implementations: 

1. `LLMGuardRuntime`: This runtime evaluates the risk level of functions using a language model. It ensures that potentially harmful functions are assessed and controlled before execution. The runtime uses a prompt-based approach to determine the risk score of a function based on its description and parameters.

2. `RemoteHttpRuntime`: This runtime allows functions to be executed on a remote HTTP server. It is useful for distributing workloads across different servers and ensuring that functions run in a controlled remote environment. The runtime supports custom entry points and arguments, and provides mechanisms to check server status and wait for readiness.

3. `DockerRuntime`: This runtime provides a containerized environment for executing functions using Docker. It ensures that functions run in isolated and reproducible environments, making it ideal for complex workflows and dependency management. The runtime supports mounting directories, copying files to containers, and adding tasks to execute commands inside containers.

Each of these runtimes provides a unique environment for executing functions, ensuring safety, remote execution, and containerized execution, respectively.

## Introduction



The `BaseRuntime` class is the base class for runtime environments used in the CAMEL system. It is designed to provide a consistent structure for executing functions in different environments and allows for easy extension to support various runtime implementations. The `BaseRuntime` class ensures that functions are executed in a controlled and predictable manner, providing mechanisms for initialization, execution, and cleanup.

### Installation

Ensure you have CAMEL AI installed in your Python environment:

In [ ]:
!pip install "camel-ai"

If you want to try DockerRuntime, please make sure you have Docker installed in your environment.

### Setting Up API Keys

You'll need to set up your API keys for OpenAI.

In [7]:
import os
from getpass import getpass

# Prompt for the API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

## LLMGuardRuntime

Key Features:

+ Evaluates function risk using a language model.
+ Assigns risk scores based on predefined criteria.
+ Provides a mechanism to ignore certain risks if necessary.

In [2]:
from camel.runtimes.llm_guard_runtime import LLMGuardRuntime
from camel.toolkits import FunctionTool


def example_function(a, b):
    return a + b


tool = FunctionTool(example_function)
runtime = LLMGuardRuntime()
runtime.add(tool)
result = tool.func(1, 2)
print(result)

2024-12-12 23:26:56,198 - camel - INFO - Camel library logging has been configured.
2024-12-12 23:26:57,668 - camel.agents.chat_agent - WARNING - Overriding the configured tools in `BaseModelBackend` with the tools from `ChatAgent`.


/home/lxk/camel/camel/toolkits/function_tool.py:384: UserWarning: Function description is missing for 
                          example_function. This may 
                          affect the quality of tool calling.
  warnings.warn(f"""Function description is missing for
/home/lxk/camel/camel/toolkits/function_tool.py:405: UserWarning: Parameter description is missing for 
                            {}. This may affect the quality of tool 
                            calling.
  warnings.warn(f"""Parameter description is missing for


2024-12-12 23:26:59,417 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-12 23:26:59,421 - camel.agents.chat_agent - INFO - Model gpt-4o-mini, index 0, processed these messages: [{'role': 'system', 'content': "You are a function safety evaluator tasked with assessing the \npotential risk level of a given function based on both its description \nand parameters. Your goal is to determine if the function may pose any \nharm to the user's environment, such as deleting files, executing \narbitrary code, or accessing sensitive information. Pay special attention \nto the provided parameters — even if a function has the potential to be \ndangerous, its actual parameters may indicate harmless behavior, and the \nrisk level should be adjusted accordingly. Use the `function_risk` tool to \nassign a risk score to the function, based on the following criteria:\n\n- **Score 1**: No harm. This includes simple operations like mathematical \n    c

## RemoteHttpRuntime

Key Features:

+ Executes functions on a remote HTTP server.
+ Supports custom entry points and arguments.
+ Provides mechanisms to check server status and wait for readiness.

In [3]:
from camel.runtimes.remote_http_runtime import RemoteHttpRuntime
from camel.toolkits.math_toolkit import MathToolkit

runtime = (
        RemoteHttpRuntime("localhost")
        .add(MathToolkit().get_tools(), "camel.toolkits.MathToolkit")
        .build()
    )
print("Waiting for runtime to be ready...")
runtime.wait()
print("Runtime is ready.")
add, sub, mul, _, _ = runtime.get_tools()
print(f"Add 1 + 2: {add.func(1, 2)}")
print(f"Subtract 5 - 3: {sub.func(5, 3)}")
print(f"Multiply 2 * 3: {mul.func(2, 3)}")

print("Documents: ", runtime.docs)
# you can open this url in browser to see the API Endpoints
# before the runtime is stopped.
# time.sleep(60)
runtime.stop()
# call runtime.stop() if you want to stop the runtime manually
# atherwise it will be stopped automatically when the program ends

Waiting for runtime to be ready...
2024-12-12 23:30:51,108 - camel - INFO - Camel library logging has been configured.
2024-12-12 23:30:51,728 - __main__ - INFO - Modules and functions: ['camel.toolkits.MathToolkit']
2024-12-12 23:30:51,729 - __main__ - INFO - Importing camel.toolkits and function MathToolkit


INFO:     Will watch for changes in these directories: ['/home/lxk/camel/docs/cookbooks']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [674247] using WatchFiles


2024-12-12 23:30:52,167 - camel - INFO - Camel library logging has been configured.
2024-12-12 23:30:52,819 - __mp_main__ - INFO - Modules and functions: ['camel.toolkits.MathToolkit']
2024-12-12 23:30:52,820 - __mp_main__ - INFO - Importing camel.toolkits and function MathToolkit


INFO:     Started server process [674287]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


INFO:     127.0.0.1:60696 - "GET / HTTP/1.1" 404 Not Found
Runtime is ready.
INFO:     127.0.0.1:60410 - "POST /add HTTP/1.1" 200 OK
Add 1 + 2: 3
INFO:     127.0.0.1:60420 - "POST /sub HTTP/1.1" 200 OK
Subtract 5 - 3: 2
INFO:     127.0.0.1:60436 - "POST /multiply HTTP/1.1" 200 OK
Multiply 2 * 3: 6
Documents:  http://localhost:8000/docs


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [674287]
INFO:     Stopping reloader process [674247]


## DockerRuntime

Key Features:

+ Executes functions in Docker containers.
+ Supports mounting directories and copying files to containers.
+ Provides mechanisms to add tasks and execute commands inside containers.

In [ ]:
from camel.runtimes import DockerRuntime
from camel.toolkits.code_execution import CodeExecutionToolkit


# tools
toolkit = CodeExecutionToolkit(verbose=True)

# change to your own docker image
runtime = DockerRuntime("xukunliu/camel").add(
    toolkit.get_tools(),
    "camel.toolkits.CodeExecutionToolkit",
    redirect_stdout=True,
    arguments=dict(verbose=True),
)

tool = runtime.get_tools()[0]

with runtime as r:
    print("Waiting for runtime to be ready...")
    r.wait()
    print("Runtime is ready.")
    tool.func("2 ** 10")

print("DockerRuntime stopped")
    

2024-12-12 23:34:31,918 - camel.runtimes.docker_runtime - INFO - Copying /home/lxk/camel/camel/runtimes/api.py to /home
2024-12-12 23:34:31,983 - camel.runtimes.docker_runtime - INFO - Container started on port 8000
Waiting for runtime to be ready...
Runtime is ready.
Executed the code below:
```py
2 ** 10
```
> Executed Results:
1024

2024-12-12 23:34:46,676 - camel.runtimes.docker_runtime - INFO - Removing container.


## Use `Runtimes` with Agent

In this section, we will demonstrate how to use the `Runtime` with an agent in the CAMEL system. 

The `Runtime` class provides an `add` function that allows you to add `FunctionTool` instances to the runtime and use `get_tools` to retrieve the wrapped functions. These functions behave exactly like the original functions, and you can add them to the agent just like any other tools.

In [9]:
from colorama import Fore
from camel.agents import ChatAgent
from camel.configs import ChatGPTConfig
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.runtimes import DockerRuntime
from camel.toolkits.code_execution import CodeExecutionToolkit
from camel.types import ModelPlatformType, ModelType
from camel.utils import print_text_animated

# Initialize the CodeExecutionToolkit
toolkit = CodeExecutionToolkit(verbose=True)

# Initialize the DockerRuntime with a specified Docker image
runtime = DockerRuntime("xukunliu/camel").add(
    toolkit.get_tools(),
    "camel.toolkits.CodeExecutionToolkit",
    redirect_stdout=True,
)

# Retrieve the tools from the runtime
tools = runtime.get_tools()

# Configure the language model
assistant_model_config = ChatGPTConfig(
    temperature=0.0,
)

# Create the model using the ModelFactory
model = ModelFactory.create(
    model_platform=ModelPlatformType.DEFAULT,
    model_type=ModelType.GPT_4O,
    model_config_dict=assistant_model_config.as_dict(),
)

# Create the system message for the agent
assistant_sys_msg = BaseMessage.make_assistant_message(
    role_name="Teacher",
    content=(
        "You are a personal math tutor and programmer. "
        "When asked a math question, "
        "write and run Python code to answer the question."
    ),
)

# Initialize the ChatAgent with the system message, model, and tools
agent = ChatAgent(
    assistant_sys_msg,
    model,
    tools=tools,
)
agent.reset()

# Use the runtime to execute the task
with runtime as r:
    r.wait()
    prompt = (
        "Weng earns $12 an hour for babysitting. "
        "Yesterday, she just did 51 minutes of babysitting. How much did she earn?"
    )
    user_msg = BaseMessage.make_user_message(role_name="User", content=prompt)
    print(Fore.YELLOW + f"user prompt:\n{prompt}\n")

    # Get the agent's response to the user message
    response = agent.step(user_msg)
    for msg in response.msgs:
        print_text_animated(Fore.GREEN + f"Agent response:\n{msg.content}\n")

2024-12-12 23:54:28,291 - camel.agents.chat_agent - WARNING - Overriding the configured tools in `BaseModelBackend` with the tools from `ChatAgent`.
2024-12-12 23:54:30,513 - camel.runtimes.docker_runtime - INFO - Copying /home/lxk/camel/camel/runtimes/api.py to /home
2024-12-12 23:54:30,605 - camel.runtimes.docker_runtime - INFO - Container started on port 8000
user prompt:
Weng earns $12 an hour for babysitting. Yesterday, she just did 51 minutes of babysitting. How much did she earn?

2024-12-12 23:54:38,240 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-12 23:54:38,304 - camel.agents.chat_agent - INFO - Model gpt-4o, index 0, processed these messages: [{'role': 'system', 'content': 'You are a personal math tutor and programmer. When asked a math question, write and run Python code to answer the question.'}, {'role': 'user', 'content': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 51 minutes of babysitting.

In this session, we introduced the BaseRuntime class and its specific runtime implementations. These components play essential roles in the CAMEL system, facilitating the execution of functions in various environments with safety, remote execution, and containerized execution.